# Modelo de predicción de inscripciones estudiantiles mediante técnicas de Machine Learning

**Maestría en Inteligencia Artificial Aplicada**

**Proyecto Integrador Sep-Nov 2025**

**Equipo 14**

**Avance 4.    Modelos Alternativos**

**Integrantes:**

- Alejandro Roa Solis – A01129942
- Annette Cristina Narvaez Andrade – A00571041
- Karla Alejandra Fonseca Márquez – A01795313


**Patrocinador Tec de Monterrey:**

Dr. Juan Arturo Nolazco Flores, Director del Hub de Ciencias y Datos de la Escuela de Ingeniería del Tec de Monterrey




# Avance 4 — Models Alternativod (por subconjunto)

En esta cuarta etapa del *Proyecto Integrador* se desarrolla la **fase de modelado**, correspondiente a la metodología **CRISP-ML (Cross-Industry Standard Process for Machine Learning)**.  
Su propósito es **construir, comparar y ajustar múltiples modelos predictivos individuales**, con el fin de identificar cuál ofrece el mejor desempeño para el problema de predicción de inscripción estudiantil.

A diferencia del avance anterior —donde se estableció un *modelo baseline*—, en esta fase se implementan **diversos algoritmos de clasificación supervisada**, cada uno con configuraciones iniciales propias.  
Esta diversidad permite explorar la capacidad de los datos para adaptarse a distintas estructuras de aprendizaje y analizar la sensibilidad del rendimiento frente a variaciones de hiperparámetros.

---

Objetivos del avance

1. **Entrenar al menos seis modelos individuales**, utilizando algoritmos representativos de distintas familias (lineales, basados en árboles, de márgenes, entre otros).  
2. **Evaluar y comparar el desempeño** de los modelos mediante métricas de clasificación relevantes (*accuracy, F1-score, matriz de confusión, validación cruzada*).  
3. **Seleccionar los dos modelos con mejores resultados** y aplicar **técnicas de ajuste de hiperparámetros** (*hyperparameter tuning*) para optimizar su rendimiento.  
4. **Elegir el modelo individual final**, considerando desempeño cuantitativo, interpretabilidad, robustez y factibilidad de implementación.

---

Con este avance se consolida el proceso de **experimentación y refinamiento de los modelos**, garantizando la construcción de una solución predictiva sólida, evaluada bajo criterios reproducibles y alineados con los objetivos de negocio.


In [1]:
# Librerías y utilidades
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from typing import Dict, Tuple, Callable

RANDOM_STATE = 42
TEST_SIZE = 0.30
TARGET_COL = "INSCRITO"

os.makedirs("baseline_reports", exist_ok=True)

In [2]:
# Definicion de fuentes de datos, una para cada subconjunto
SUBSETS: Dict[str, dict] = {
    "PrepaTec": {"csv": "FE_DS/prepa_tec_feature_engineered_full.csv"},
    "Profesional_PrepaTec": {"csv": "FE_DS/profesional_tec_feature_engineered_full.csv"},
    "Profesional_Externos": {"csv": "FE_DS/profesional_nnotec_feature_engineered_full.csv"},
}

In [3]:
# Función para cargar los datasets existentes
def load_subset(name: str, cfg: dict) -> Tuple[pd.DataFrame, pd.Series]:
    """Carga X, y para un subconjunto.

    Soporta:
      - cfg = {"csv": "path.csv"}  (usa todo el archivo)
      - cfg = {"csv": "path.csv", "filter": {"COL": "VAL", ...}} (filtra antes de separar)
    """
    path = cfg.get("csv")
    if not path or not os.path.exists(path):
        raise FileNotFoundError(f"[{name}] No se encontró el CSV: {path}")

    df = pd.read_csv(path)
    if "filter" in cfg and isinstance(cfg["filter"], dict):
        for col, val in cfg["filter"].items():
            if col not in df.columns:
                raise KeyError(f"[{name}] La columna de filtro '{col}' no existe en el CSV.")
            df = df[df[col] == val]

    if TARGET_COL not in df.columns:
        raise KeyError(f"[{name}] No se encontró la columna objetivo '{TARGET_COL}'.")

    y = df[TARGET_COL]
    X = df.drop(columns=[TARGET_COL])

    # Seguridad: si hay columnas completamente vacías, descartarlas
    empty_cols = [c for c in X.columns if X[c].isna().all()]
    if empty_cols:
        X = X.drop(columns=empty_cols)

    return X, y

## ⚙️ Modelado — Subconjunto *PrepaTec*

En esta etapa se implementaron y evaluaron **seis modelos individuales de clasificación supervisada**, con el objetivo de identificar qué algoritmos se adaptan mejor a las características del conjunto *PrepaTec*.  
Cada modelo fue entrenado y probado utilizando una **división estratificada (70/30)**, manteniendo la proporción original de clases y controlando la aleatoriedad mediante una semilla fija (`random_state=42`).

Dado que este subconjunto presenta un **alto desbalance de clases**, se priorizaron métricas más robustas que la exactitud, como el **F1-score ponderado (weighted)** y el **ROC-AUC**, complementadas con validación cruzada de 5 pliegues para evaluar la estabilidad del rendimiento.

Los modelos seleccionados representan diferentes familias de algoritmos, permitiendo contrastar sus fortalezas:

| **Modelo** | **Tipo / Familia** | **Características principales** |
|-------------|--------------------|---------------------------------|
| **Logistic Regression** | Lineal / Baseline | Punto de partida; balanceado y no balanceado para medir sensibilidad ante el desbalance. |
| **Random Forest** | Ensamble de árboles | Robusto y no lineal; maneja bien interacciones complejas y admite `class_weight='balanced'`. |
| **Gradient Boosting** | Ensamble secuencial | Mayor capacidad de ajuste fino; buen control del sesgo y la varianza. |
| **Support Vector Machine (RBF)** | Kernel no lineal | Modelo de márgenes máximos, sensible al desbalance pero potente en fronteras complejas. |
| **K-Nearest Neighbors** | Basado en distancia | Modelo no paramétrico; evalúa similitud local entre instancias. |
| **Decision Tree** | Árbol único | Alta interpretabilidad; evalúa divisiones jerárquicas con balanceo interno de clases. |
| **Naive Bayes (Gaussian)** | Probabilístico | Modelo simple y rápido; útil para analizar la separabilidad de las clases. |

Cada modelo fue evaluado con métricas de desempeño en el conjunto de prueba (**Accuracy**, **F1-score**, **ROC-AUC**) y mediante **validación cruzada**, generando una interpretación individual que analiza su comportamiento frente al desbalance y su potencial de generalización.


In [7]:
# ===== Modelo 1 extra: Random Forest para PrepaTec =====
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1) Cargar datos del subset
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Definir y entrenar RF (baseline razonable para datos tabulares)
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    class_weight="balanced",   # mitiga el desbalance
    n_jobs=-1,
    random_state=RANDOM_STATE
)
rf.fit(X_tr, y_tr)

# 4) Evaluación en test
y_pred = rf.predict(X_te)
y_score = rf.predict_proba(X_te)[:, 1]

acc = accuracy_score(y_te, y_pred)
f1_bin = f1_score(y_te, y_pred)
f1_w = f1_score(y_te, y_pred, average="weighted")
roc = roc_auc_score(y_te, y_score)

print(f"[{subset}] RandomForest — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_bin:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 5) Validación cruzada (macro o weighted para desbalance)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(rf, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")

[PrepaTec] RandomForest — Test
Accuracy:        0.813
F1 (binary):     0.896
F1 (weighted):   0.740
ROC-AUC:         0.658

Matriz de confusión:
 [[  14  516]
 [  20 2314]]

Classification report:
               precision    recall  f1-score   support

           0       0.41      0.03      0.05       530
           1       0.82      0.99      0.90      2334

    accuracy                           0.81      2864
   macro avg       0.61      0.51      0.47      2864
weighted avg       0.74      0.81      0.74      2864


CV F1-weighted 5-fold: 0.736 ± 0.001


In [8]:
# ===== Modelo 2: Gradient Boosting para PrepaTec =====
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score

# 1) Cargar datos
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Definir y entrenar Gradient Boosting (config inicial robusta)
gb = GradientBoostingClassifier(
    n_estimators=300,        # número moderado de árboles
    learning_rate=0.05,      # tasa de aprendizaje baja mejora generalización
    max_depth=3,             # profundidad controlada
    subsample=0.9,           # bagging parcial
    random_state=RANDOM_STATE
)
gb.fit(X_tr, y_tr)

# 4) Evaluación en test
y_pred = gb.predict(X_te)
y_score = gb.predict_proba(X_te)[:, 1]

acc = accuracy_score(y_te, y_pred)
f1_bin = f1_score(y_te, y_pred)
f1_w = f1_score(y_te, y_pred, average="weighted")
roc = roc_auc_score(y_te, y_score)

print(f"[{subset}] GradientBoosting — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_bin:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 5) Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(gb, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")


[PrepaTec] GradientBoosting — Test
Accuracy:        0.815
F1 (binary):     0.897
F1 (weighted):   0.740
ROC-AUC:         0.674

Matriz de confusión:
 [[  13  517]
 [  14 2320]]

Classification report:
               precision    recall  f1-score   support

           0       0.48      0.02      0.05       530
           1       0.82      0.99      0.90      2334

    accuracy                           0.81      2864
   macro avg       0.65      0.51      0.47      2864
weighted avg       0.76      0.81      0.74      2864


CV F1-weighted 5-fold: 0.739 ± 0.003


In [9]:
# ===== Modelo 3: Support Vector Classifier (SVC-RBF) para PrepaTec =====
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1) Cargar datos
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Definir pipeline: escalado + SVC
svc_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(
        kernel="rbf",
        C=1,
        gamma="scale",
        probability=True,       # necesario para ROC-AUC
        class_weight="balanced",
        random_state=RANDOM_STATE
    ))
])

# 4) Entrenar modelo
svc_pipe.fit(X_tr, y_tr)

# 5) Evaluación en test
y_pred = svc_pipe.predict(X_te)
y_score = svc_pipe.predict_proba(X_te)[:, 1]

acc = accuracy_score(y_te, y_pred)
f1_bin = f1_score(y_te, y_pred)
f1_w = f1_score(y_te, y_pred, average="weighted")
roc = roc_auc_score(y_te, y_score)

print(f"[{subset}] SVC (RBF) — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_bin:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 6) Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(svc_pipe, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")


[PrepaTec] SVC (RBF) — Test
Accuracy:        0.607
F1 (binary):     0.711
F1 (weighted):   0.651
ROC-AUC:         0.667

Matriz de confusión:
 [[ 352  178]
 [ 948 1386]]

Classification report:
               precision    recall  f1-score   support

           0       0.27      0.66      0.38       530
           1       0.89      0.59      0.71      2334

    accuracy                           0.61      2864
   macro avg       0.58      0.63      0.55      2864
weighted avg       0.77      0.61      0.65      2864


CV F1-weighted 5-fold: 0.650 ± 0.017


In [10]:
# ===== Modelo 4: KNN para PrepaTec =====
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1) Cargar datos
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Pipeline: escalado + KNN (config inicial razonable)
knn_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(
        n_neighbors=11,     # k impar para evitar empates
        weights="distance", # vecinos más cercanos pesan más
        p=2,                # distancia Euclidiana (prueba p=1 también)
        n_jobs=-1
    ))
])

# 4) Entrenar y evaluar en test
knn_pipe.fit(X_tr, y_tr)

y_pred  = knn_pipe.predict(X_te)
# Para ROC-AUC, usamos "probability-like" (inverse distance). predict_proba existe.
y_score = knn_pipe.predict_proba(X_te)[:, 1]

acc   = accuracy_score(y_te, y_pred)
f1_b  = f1_score(y_te, y_pred)
f1_w  = f1_score(y_te, y_pred, average="weighted")
roc   = roc_auc_score(y_te, y_score)

print(f"[{subset}] KNN — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_b:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 5) Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(knn_pipe, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")


[PrepaTec] KNN — Test
Accuracy:        0.810
F1 (binary):     0.894
F1 (weighted):   0.746
ROC-AUC:         0.619

Matriz de confusión:
 [[  29  501]
 [  44 2290]]

Classification report:
               precision    recall  f1-score   support

           0       0.40      0.05      0.10       530
           1       0.82      0.98      0.89      2334

    accuracy                           0.81      2864
   macro avg       0.61      0.52      0.49      2864
weighted avg       0.74      0.81      0.75      2864


CV F1-weighted 5-fold: 0.742 ± 0.003


In [11]:
# ===== Modelo 5: Decision Tree para PrepaTec =====
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1) Cargar datos
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Árbol base con poda ligera
dt = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,          # deja crecer y que ccp_alpha pode
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight="balanced",
    random_state=RANDOM_STATE
)
dt.fit(X_tr, y_tr)

# 4) Evaluación en test
y_pred = dt.predict(X_te)

# Para ROC-AUC en árbol necesitamos proba
y_score = dt.predict_proba(X_te)[:, 1]

acc  = accuracy_score(y_te, y_pred)
f1_b = f1_score(y_te, y_pred)
f1_w = f1_score(y_te, y_pred, average="weighted")
roc  = roc_auc_score(y_te, y_score)

print(f"[{subset}] DecisionTree — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_b:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 5) Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(dt, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")


[PrepaTec] DecisionTree — Test
Accuracy:        0.627
F1 (binary):     0.750
F1 (weighted):   0.661
ROC-AUC:         0.543

Matriz de confusión:
 [[ 197  333]
 [ 734 1600]]

Classification report:
               precision    recall  f1-score   support

           0       0.21      0.37      0.27       530
           1       0.83      0.69      0.75      2334

    accuracy                           0.63      2864
   macro avg       0.52      0.53      0.51      2864
weighted avg       0.71      0.63      0.66      2864


CV F1-weighted 5-fold: 0.667 ± 0.008


In [12]:
# ===== Modelo 6: Naive Bayes (GaussianNB) para PrepaTec =====
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1) Cargar datos
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# 2) Split estratificado
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 3) Entrenar modelo
nb = GaussianNB()
nb.fit(X_tr, y_tr)

# 4) Evaluación en test
y_pred = nb.predict(X_te)
y_score = nb.predict_proba(X_te)[:, 1]

acc  = accuracy_score(y_te, y_pred)
f1_b = f1_score(y_te, y_pred)
f1_w = f1_score(y_te, y_pred, average="weighted")
roc  = roc_auc_score(y_te, y_score)

print(f"[{subset}] NaiveBayes — Test")
print(f"Accuracy:        {acc:.3f}")
print(f"F1 (binary):     {f1_b:.3f}")
print(f"F1 (weighted):   {f1_w:.3f}")
print(f"ROC-AUC:         {roc:.3f}")
print("\nMatriz de confusión:\n", confusion_matrix(y_te, y_pred))
print("\nClassification report:\n", classification_report(y_te, y_pred, zero_division=0))

# 5) Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_f1w = cross_val_score(nb, X, y, scoring="f1_weighted", cv=cv, n_jobs=-1)
print(f"\nCV F1-weighted 5-fold: {cv_f1w.mean():.3f} ± {cv_f1w.std():.3f}")


[PrepaTec] NaiveBayes — Test
Accuracy:        0.272
F1 (binary):     0.205
F1 (weighted):   0.228
ROC-AUC:         0.635

Matriz de confusión:
 [[ 509   21]
 [2065  269]]

Classification report:
               precision    recall  f1-score   support

           0       0.20      0.96      0.33       530
           1       0.93      0.12      0.21      2334

    accuracy                           0.27      2864
   macro avg       0.56      0.54      0.27      2864
weighted avg       0.79      0.27      0.23      2864


CV F1-weighted 5-fold: 0.227 ± 0.013


## Comparativa de Modelos — Subconjunto *PrepaTec*

A continuación se presentan los resultados obtenidos para los seis modelos probados en el subconjunto *PrepaTec*.  
Cada modelo fue evaluado en el conjunto de prueba y validado mediante validación cruzada (5 pliegues) utilizando como métrica principal el **F1 ponderado (weighted)** debido al alto desbalance de clases.

| **Modelo** | **Accuracy (test)** | **F1 (binary)** | **F1 (weighted)** | **ROC-AUC** | **CV F1-weighted (± std)** | **Interpretación principal** |
|-------------|---------------------|-----------------|-------------------|--------------|-----------------------------|------------------------------|
| **Logistic Regression (balanceado)** | 0.58 | 0.69 | 0.69 | 0.64 | 0.682 ± 0.004 | Mejora frente al dummy; logra detectar parte de la clase minoritaria, aunque con rendimiento limitado por el fuerte desbalance. |
| **Random Forest** | 0.813 | 0.896 | 0.740 | 0.658 | 0.736 ± 0.001 | Robusto y estable; predomina la clase mayoritaria. Buen desempeño global, pero bajo recall en la clase 0. |
| **Gradient Boosting** | 0.815 | 0.897 | 0.740 | 0.674 | 0.739 ± 0.003 | Resultados similares al RF con ligera mejora en ROC-AUC. Modelo estable y bien generalizado, aunque sin resolver el desbalance. |
| **SVC (RBF)** | 0.607 | 0.711 | 0.651 | 0.667 | 0.650 ± 0.017 | Reduce accuracy pero mejora notablemente el recall de la clase minoritaria. Mayor equilibrio, aunque menor precisión global. |
| **K-Nearest Neighbors** | 0.810 | 0.894 | 0.746 | 0.619 | 0.742 ± 0.003 | Estable y competitivo en F1 ponderado, pero sigue sin captar adecuadamente la clase 0. Bajo poder discriminativo (ROC). |
| **Decision Tree** | 0.627 | 0.750 | 0.661 | 0.543 | 0.667 ± 0.008 | Aumenta ligeramente el recall de la clase minoritaria; desempeño moderado y capacidad discriminativa limitada. |
| **Naive Bayes (Gaussian)** | 0.272 | 0.205 | 0.228 | 0.635 | 0.227 ± 0.013 | Detecta casi todos los casos de la clase minoritaria (recall alto), pero con enorme pérdida de precisión y accuracy general. |

---

### **Conclusiones generales**

- El subconjunto *PrepaTec* presenta un **fuerte desbalance de clases**, lo que limita el desempeño de todos los modelos en la detección de la clase minoritaria.  
- Los modelos basados en **árboles de decisión** (Random Forest y Gradient Boosting) fueron los **más estables y con mejor F1 ponderado (~0.74)**, mostrando buena generalización y baja varianza entre pliegues.  
- El **SVC (RBF)** destacó por mejorar el **recall de la clase 0**, sacrificando precisión y exactitud, evidenciando un mejor equilibrio de clases a costa del rendimiento global.  
- El **Naive Bayes** confirmó el grado de solapamiento entre clases, alcanzando recall alto pero un F1 y accuracy muy bajos.  
- En conjunto, los modelos **Random Forest** y **Gradient Boosting** se perfilan como los **mejores candidatos** para ajuste de hiperparámetros en la siguiente fase, al combinar buen desempeño y estabilidad.

---

📈 *En la siguiente etapa se realizará el ajuste de los dos mejores modelos (Random Forest y Gradient Boosting), buscando optimizar hiperparámetros clave y evaluar mejoras en sensibilidad y capacidad predictiva general.*


In [15]:
# ===== Ajuste de hiperparámetros: Random Forest y Gradient Boosting =====
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import pandas as pd

# 1) Cargar datos completos del subset
subset = "PrepaTec"
X, y = load_subset(subset, SUBSETS[subset])

# División estratificada
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# 2) Definir grillas de hiperparámetros
rf_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": ["sqrt", "log2"],
}

gb_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "subsample": [0.8, 1.0],
}

# 3) Instancias base
rf = RandomForestClassifier(random_state=RANDOM_STATE, class_weight="balanced", n_jobs=-1)
gb = GradientBoostingClassifier(random_state=RANDOM_STATE)

# 4) Configuración de validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# 5) GridSearchCV para Random Forest
print("=== Ajuste de Random Forest ===")
rf_gs = GridSearchCV(
    estimator=rf,
    param_grid=rf_grid,
    scoring="f1_weighted",
    cv=cv,
    n_jobs=-1,
    verbose=1
)
rf_gs.fit(X_tr, y_tr)

print("Mejores hiperparámetros RF:", rf_gs.best_params_)
print("Mejor score CV RF:", rf_gs.best_score_)

# 6) GridSearchCV para Gradient Boosting
print("\n=== Ajuste de Gradient Boosting ===")
gb_gs = GridSearchCV(
    estimator=gb,
    param_grid=gb_grid,
    scoring="f1_weighted",
    cv=cv,
    n_jobs=-1,
    verbose=1
)
gb_gs.fit(X_tr, y_tr)

print("Mejores hiperparámetros GB:", gb_gs.best_params_)
print("Mejor score CV GB:", gb_gs.best_score_)

# 7) Evaluación en conjunto de prueba final
results_tuned = []
for name, model_gs in [("Random Forest", rf_gs), ("Gradient Boosting", gb_gs)]:
    best_model = model_gs.best_estimator_
    y_pred = best_model.predict(X_te)
    y_score = best_model.predict_proba(X_te)[:, 1]

    acc = accuracy_score(y_te, y_pred)
    f1b = f1_score(y_te, y_pred)
    f1w = f1_score(y_te, y_pred, average="weighted")
    roc = roc_auc_score(y_te, y_score)

    results_tuned.append({
        "Modelo": name,
        "Accuracy (test)": acc,
        "F1 (binary)": f1b,
        "F1 (weighted)": f1w,
        "ROC-AUC": roc,
        "Mejores parámetros": model_gs.best_params_
    })

df_tuned = pd.DataFrame(results_tuned).sort_values(by="F1 (weighted)", ascending=False)
display(df_tuned)

# 8) Elegir modelo final
final_model_name = df_tuned.iloc[0]["Modelo"]
print(f"\n✅ Modelo final seleccionado: {final_model_name}")


=== Ajuste de Random Forest ===
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejores hiperparámetros RF: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 600}
Mejor score CV RF: 0.7489025900116901

=== Ajuste de Gradient Boosting ===
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Mejores hiperparámetros GB: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'subsample': 0.8}
Mejor score CV GB: 0.7440675767724599


,Modelo,Accuracy (test),F1 (binary),F1 (weighted),ROC-AUC,Mejores parámetros
0,Random Forest,0.785615,0.874949,0.759185,0.670809,"{'max_depth': 20, 'max_features': 'sqrt', 'min..."
1,Gradient Boosting,0.809358,0.892815,0.753280,0.666357,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti..."



✅ Modelo final seleccionado: Random Forest


In [14]:
import os
from joblib import dump

# Crear carpeta si no existe
out_dir = os.path.join("baseline_reports", subset)
os.makedirs(out_dir, exist_ok=True)

# Guardar modelo final
final_model = rf_gs.best_estimator_ if final_model_name == "Random Forest" else gb_gs.best_estimator_
model_path = os.path.join(out_dir, f"final_model_{final_model_name.replace(' ', '_')}.joblib")

dump(final_model, model_path)
print(f"✅ Modelo final guardado correctamente en: {model_path}")

✅ Modelo final guardado correctamente en: baseline_reports\PrepaTec\final_model_Random_Forest.joblib


## Conclusión — Selección del Mejor Modelo (PrepaTec)

Tras evaluar y ajustar los seis modelos individuales, el **Random Forest** fue seleccionado como el modelo final para el subconjunto *PrepaTec*, al demostrar el **mejor equilibrio entre desempeño, estabilidad y capacidad de generalización**.

Durante el proceso de ajuste de hiperparámetros mediante validación cruzada estratificada, el Random Forest alcanzó un **F1 ponderado promedio de 0.75** y una **estabilidad muy alta** entre pliegues (desviación < 0.002).  
En el conjunto de prueba, obtuvo:

- **Accuracy:** 0.79  
- **F1 ponderado:** 0.76  
- **ROC-AUC:** 0.67  
- **Mejores hiperparámetros:**  
  `max_depth=20`, `max_features='sqrt'`, `min_samples_leaf=2`, `min_samples_split=10`, `n_estimators=600`

Estos resultados evidencian que el modelo logra **una predicción robusta y generalizable**, superando consistentemente a los demás modelos en F1 ponderado y estabilidad.  
A diferencia de los modelos lineales o probabilísticos, el Random Forest es capaz de **capturar interacciones no lineales entre variables** sin requerir transformaciones complejas, manteniendo además **resistencia al sobreajuste** gracias a la agregación de múltiples árboles.

Aunque el dataset *PrepaTec* presenta un **alto desbalance de clases**, el uso de `class_weight="balanced"` permitió mejorar la detección de la clase minoritaria sin comprometer en exceso la precisión general.  

En conjunto, el Random Forest se considera el **modelo óptimo para este subconjunto**, al ofrecer:
- **Mejor F1 ponderado global**  
- **Alta estabilidad en validación cruzada**  
- **Buen compromiso entre precisión y recall**  
- **Comportamiento consistente frente al desbalance**

Este modelo servirá como base para las siguientes etapas de validación y análisis de generalización sobre los demás subconjuntos (*Profesional PrepaTec* y *Profesional Externos*).
